# Forte Sets: Complete PC-Set Universe

This notebook builds a **complete dataset of all 4096 subsets** of the 12 pitch classes {0, 1, ..., 11}, along with various **relationship links** between them.

## What's Here

### Nodes DataFrame (4096 rows)
Each row represents one subset, identified by its **bitmap integer** (0 to 4095).
- `id_`: Bitmap integer where bit `i` is set iff pitch class `i` is in the set
- `pcset`: Tuple representation, e.g., `(0, 4, 7)` for C major triad
- `cardinality`: Number of pitch classes
- `prime_form`: Canonical representative under T/I equivalence
- `forte_name`: Forte label (e.g., "3-11") if applicable
- `interval_vector`: 6-tuple counting interval classes

### Link DataFrames
- **Immediate subset links**: Hasse diagram of the subset lattice
- **Complement links**: Each set paired with its complement
- **TI-equivalence links**: Sets sharing the same prime form (set class)
- **Z-relation links**: Sets with same interval vector but different prime form
- **R_p similarity links**: Sets sharing n-1 elements under some T/I

### Key Concepts
- **Pitch class (pc)**: Note mod 12 (0=C, 1=C♯, ..., 11=B)
- **Prime form**: Lexicographically smallest form under T_n and I operations
- **Interval vector**: [ic1, ic2, ic3, ic4, ic5, ic6] counts
- **Z-relation**: Same interval content, different prime form
- **Set complex K/Kh**: Reciprocal inclusion relations

In [1]:
# Import PC-set theory functions from atonal.base
from atonal.base import (
    transpose,
    invert,
    best_normal_order,
    prime_form,
    interval_vector,
    is_transpositionally_symmetric,
    distinct_transpositions,
    distinct_inversions,
    max_invariance_degrees,
    combinatorial_property_hexachord,
    kh_complex_size,
    z_correspondent_prime_form,
    FORTE_CLASSES,
    PRIME_TO_FORTE,
    FORTE_TO_PRIME,
    forte_name,
    int_to_pcset as _int_to_tuple,
    pcset_to_int as _tuple_to_int,
)

# Note: Use forte_name() directly instead of get_forte_name()
# Note: Use prime_form() for equivalence checking instead of wrappers

print("✓ Imported PC-set functions from atonal.base")

✓ Imported PC-set functions from atonal.base


## Representation Converter

A universal function to translate between different representations of pc-sets:
- **`int`**: Bitmap integer (0 to 4095), where bit `i` is set if pitch class `i` is in the set
- **`tuple`**: Sorted tuple of pitch classes, e.g., `(0, 4, 7)`
- **`frozenset`** / **`set`**: Unordered collection
- **`forte`**: Forte name string, e.g., `"4-19"` (only valid for certain sets)
- **`prime`**: Prime form tuple (canonical representative of equivalence class)

In [2]:
# Import the representation converter from atonal.base
from atonal.base import pc_set_convert, int_to_pcset, pcset_to_int

# Aliases for backward compatibility
_int_to_frozenset = lambda n: frozenset(int_to_pcset(n))

# Test the converter
print("Converter tests:")
print(f"  145 -> tuple: {pc_set_convert(145, 'tuple')}")
print(f"  (0,4,7) -> int: {pc_set_convert((0,4,7), 'int')}")
print(f"  (0,4,7) -> forte: {pc_set_convert((0,4,7), 'forte')}")
print(f"  '3-11' -> prime: {pc_set_convert('3-11', 'prime')}")

Converter tests:
  145 -> tuple: (0, 4, 7)
  (0,4,7) -> int: 145
  (0,4,7) -> forte: 3-11
  '3-11' -> prime: (0, 3, 7)


## Nodes DataFrame: All 4096 Subsets

Each row represents one of the $2^{12} = 4096$ possible subsets of $\{0, 1, ..., 11\}$.

The **id** column is the bitmap integer representation.

Additional columns capture properties useful for analysis and filtering.

In [3]:
import pandas as pd
import numpy as np

# Import the nodes builder from atonal.base (already includes all new fields!)
from atonal.base import build_pcset_nodes_df

# Build the nodes dataframe
nodes_df = build_pcset_nodes_df()
print(f"Nodes DataFrame: {len(nodes_df)} rows")
print(f"\nCardinality distribution:")
print(nodes_df['cardinality'].value_counts().sort_index())
print(f"\nForte sets (prime forms): {nodes_df['is_forte_set'].sum()}")
print(f"Sets with Forte names: {nodes_df['forte_name'].notna().sum()}")


print(f"{nodes_df.shape=}\n")

nodes_df.iloc[30]  # have a look a row

Nodes DataFrame: 4096 rows

Cardinality distribution:
cardinality
0       1
1      12
2      66
3     220
4     495
5     792
6     924
7     792
8     495
9     220
10     66
11     12
12      1
Name: count, dtype: int64

Forte sets (prime forms): 114
Sets with Forte names: 3662
nodes_df.shape=(4096, 21)



id_                                           30
pcset                               (1, 2, 3, 4)
cardinality                                    4
contains_zero                              False
complement_id                               4065
prime_form                          (0, 1, 2, 3)
forte_name                                   4-1
is_forte_set                               False
interval_vector               (3, 2, 1, 0, 0, 0)
is_t_symmetric                             False
z_correspondent_prime_form                  None
z_correspondent_forte_name                  None
n_T                                           12
n_I                                           12
kh_size                                        2
hexachord_combinatorial                     None
max_T_invariance                               3
max_T_invariance_n                             1
max_I_invariance                               4
max_I_invariance_n                             5
best_normal_order   

In [4]:
major_scale_pcset = (0, 2, 4, 5, 7, 9, 11)

# find the index of nodes_df['pcset'] == major_scale_pcset


In [5]:

t = (nodes_df['pcset'] == major_scale_pcset)
# find the index of the first matching row
index = nodes_df.index[t][0]
index

np.int64(2741)

In [21]:
# Major Scale (C major) Test Suite
major_scale_pcset = (0, 2, 4, 5, 7, 9, 11)
major_scale_row = nodes_df[nodes_df['pcset'] == major_scale_pcset].iloc[0]
major_scale_row


id_                                             2741
pcset                         (0, 2, 4, 5, 7, 9, 11)
cardinality                                        7
contains_zero                                   True
complement_id                                   1354
prime_form                    (0, 1, 3, 5, 6, 8, 10)
forte_name                                      7-35
is_forte_set                                   False
interval_vector                   (2, 5, 4, 3, 6, 1)
is_t_symmetric                                 False
z_correspondent_prime_form                      None
z_correspondent_forte_name                      None
n_T                                               12
n_I                                               12
kh_size                                            2
hexachord_combinatorial                         None
max_T_invariance                                   6
max_T_invariance_n                                 5
max_I_invariance                              

In [ ]:

# Basic identity and structure
assert major_scale_row['id_'] == 2741
assert major_scale_row['pcset'] == (0, 2, 4, 5, 7, 9, 11)
assert major_scale_row['cardinality'] == 7
assert major_scale_row['contains_zero'] == True

# Complementation
assert major_scale_row['complement_id'] == 1354
# Complement is {1, 3, 6, 8, 10} - the pentatonic collection!

# Set-class identity
assert major_scale_row['prime_form'] == (0, 1, 3, 5, 6, 8, 10)
assert major_scale_row['forte_name'] == '7-35'
assert major_scale_row['is_forte_set'] == False  # Not the canonical prime form representative

# Interval content
assert major_scale_row['interval_vector'] == (2, 5, 4, 3, 6, 1)

# Z-relations
assert major_scale_row['is_t_symmetric'] == False
assert major_scale_row['z_correspondent_prime_form'] is None
assert major_scale_row['z_correspondent_forte_name'] is None

# Symmetry and orbit sizes
assert major_scale_row['n_T'] == 12
assert major_scale_row['n_I'] == 12

# Relational properties
assert major_scale_row['kh_size'] == 2

# Hexachord combinatoriality (N/A for cardinality 7)
assert major_scale_row['hexachord_combinatorial'] is None

# Invariance under transposition
assert major_scale_row['max_T_invariance'] == 6
assert major_scale_row['max_T_invariance_n'] == 5

# Invariance under inversion - THE BIG DISCOVERY!
assert major_scale_row['max_I_invariance'] == 7
assert major_scale_row['max_I_invariance_n'] == 4

# Normal order representation
assert major_scale_row['best_normal_order'] == (11, 0, 2, 4, 5, 7, 9)

## 🎼 Musical Commentary: The Hidden Geometry of the Major Scale

### **The Diatonic Collection as Set Class 7-35**

The major scale {0,2,4,5,7,9,11} and natural minor scale {0,1,3,5,6,8,10} are **inversionally equivalent**—they belong to the same Forte set class 7-35. This mathematical fact captures what musicians have always known intuitively: major and minor are two sides of the same coin, related by a "mirroring" operation. The major scale's bright character and the minor scale's dark character are inversions of each other in the deepest structural sense.

**Why `is_forte_set = False`?** The C major scale (0,2,4,5,7,9,11) is *not* the prime form representative. The prime form (0,1,3,5,6,8,10) corresponds to A natural minor starting on A, which happens to be lexicographically "smaller" after applying all transposition-inversion operations. This is purely a cataloging convention—both scales are equally "fundamental."

---

### **Interval Vector (2,5,4,3,6,1): The Sonic Fingerprint**

The interval vector tells us the **harmonic content** of the major scale:

- **ic1 (minor 2nd): 2 instances** — The scale contains two half-steps: E-F and B-C. These "tendency tones" create the scale's characteristic tension and resolution.

- **ic2 (major 2nd): 5 instances** — Five whole steps give the scale its stepwise, singable quality. This is why melodies predominantly move by step.

- **ic3 (minor 3rd): 4 instances** — Four minor thirds scattered through the scale. Notice: D-F, E-G, A-C, B-D—these form the building blocks of the three minor triads (ii, iii, vi) in the harmonization of the scale.

- **ic4 (major 3rd): 3 instances** — Three major thirds: C-E, F-A, G-B—these form the three major triads (I, IV, V) in tonal harmony!

- **ic5 (perfect 4th): 6 instances** — Six perfect fourths. This abundance explains why fourths are the "consonant gaps" in melody—skipping by fourth feels natural because the scale is saturated with this interval.

- **ic6 (tritone): 1 instance** — Exactly one tritone: F-B. This unique interval is the **leading tone to tonic** relationship that defines functional tonality. Its singularity makes it the most structurally important interval in the scale.

**Musical insight**: The interval vector explains why certain harmonies are "diatonic" (built entirely from the scale) while others are "chromatic" (require notes outside the scale). The major scale is remarkably balanced—neither too dissonant (only 1 tritone) nor too consonant (plenty of tension-creating minor seconds).

---

### **n_T = 12, n_I = 12: Maximum Transpositional Diversity**

These values tell us that all 12 transpositions of the major scale are **distinct**, and all 12 inversions (relative to different axes) are **distinct**. This means:

- **12 major keys**: C major, C♯ major, D major... all sound different (use different pitch-class collections)
- **12 minor keys**: A minor, B♭ minor, B minor... all sound different

This is not true for all scales! For example:
- The **whole-tone scale** {0,2,4,6,8,10} has `n_T = 2` — there are only TWO distinct whole-tone scales
- The **diminished scale** {0,1,3,4,6,7,9,10} has `n_T = 3` — only THREE distinct diminished scales

The major scale's `n_T = 12` gives Western tonal music its **key diversity**—composers have 12 unique tonal centers to explore, each with its own palette of available chords and melodies.

---

### **max_I_invariance = 7: Perfect Inversional Symmetry!**

This is the most profound discovery: **All 7 notes of the major scale remain in the scale when inverted around axis 4** (the E/E♭ axis in C major).

Let's see the inversion I₄(x) = (4 - x) mod 12:

```
C (0)  → E (4)   ✓ in scale
D (2)  → D (2)   ✓ fixed point!
E (4)  → C (0)   ✓ in scale
F (5)  → B (11)  ✓ in scale
G (7)  → A (9)   ✓ in scale
A (9)  → G (7)   ✓ in scale
B (11) → F (5)   ✓ in scale
```

**Musical interpretation**: The major scale is **perfectly balanced around its mediant** (the 3rd scale degree, E in C major). This geometric symmetry has deep musical consequences:

1. **Tonic-Dominant Balance**: C and G are equidistant from E (4 semitones away). This explains the "equilibrium" between tonic and dominant in classical harmony.

2. **Subdominant-Supertonic Balance**: F and D are equidistant from E (3 semitones away). The IV and ii chords have complementary functions.

3. **The Fixed Point D**: D maps to itself! This gives the ii chord its "pivotal" quality in progressions—it's the harmonic fulcrum of the scale.

4. **Leading Tone and Subdominant**: B and F (the tritone pair!) swap positions under this inversion, explaining their complementary roles as tendency tones.

This symmetry is why **the major scale sounds so balanced and complete** to Western ears—it's mathematically perfect!

---

### **max_T_invariance = 6, max_T_invariance_n = 5**

Six pitch classes remain fixed when transposing by **5 semitones (a perfect fourth)**. Why?

If we transpose C major up a perfect fourth (5 semitones), we get F major: {5,7,9,10,0,2,4} = {F,G,A,B♭,C,D,E}.

Comparing:
- **C major**: {C, D, E, F, G, A, B}
- **F major**: {F, G, A, B♭, C, D, E}

Shared notes: **{C, D, E, F, G, A}** — 6 out of 7!

**Musical insight**: This explains the **circle of fifths** and why modulating by fourths/fifths sounds smooth—you change only **one note** (B becomes B♭). This is the foundation of tonal modulation in common-practice music. Beethoven, Mozart, and Bach exploit this property constantly.

---

### **kh_size = 2: Minimal Set-Complex**

The Kh complex measures how many pitch-class sets are related to the major scale through **reciprocal subset/superset relations** (considering complements).

`kh_size = 2` means only two sets satisfy the condition:
1. **∅** (empty set)
2. **Chromatic collection** (all 12 notes)

**Why so small?** The major scale {0,2,4,5,7,9,11} and its complement {1,3,6,8,10} are **disjoint**—they share no notes! This means:
- No non-trivial set can be a subset of **both** the scale and its complement
- Only the empty set (subset of everything) and the chromatic collection (superset of everything) satisfy the reciprocal condition

**Musical insight**: The major scale and its complement (a **pentatonic collection**) are maximally distinct—zero overlap. This explains why:
- The pentatonic scale {C♯, E♭, F♯, G♯, B♭} sounds so foreign when played against C major
- The "black keys vs. white keys" dichotomy on the piano is a complete partitioning of chromatic space
- This is why Debussy and other Impressionists could create such exotic sounds by juxtaposing diatonic and pentatonic collections

---

### **Complement {1,3,6,8,10}: The "Other" Pentatonic**

The complement_id = 1354 gives us {1,3,6,8,10} = {C♯, E♭, F♯, G♯, B♭}, which is **not** the familiar major pentatonic, but rather a different 5-note collection.

Wait—let's check what set class this is:

Actually, this is **Forte 5-35**, which is the complement of 7-35. If we compute the prime form of {1,3,6,8,10}, we'd find it's related to the major pentatonic {0,2,4,7,9} by transposition. So in a sense, the "antimatter" to the major scale is *another* pentatonic collection—just not the one we're culturally familiar with!

---

### **Conclusion: Mathematics Illuminates Music**

These assertions and the data they verify reveal that the major scale is not just a "nice-sounding collection" but a **mathematically perfect structure** with deep symmetries:

- **Perfect inversional balance** around the mediant (I₄ fixes all 7 notes)
- **Maximal key diversity** (n_T = 12 gives us 12 independent keys)
- **Optimal modulation structure** (T₅ changes only 1 note, enabling smooth key changes)
- **Balanced interval content** (rich in consonant intervals, sparing with dissonance)
- **Complementary relationship** with pentatonic collections

These properties explain why the major scale has been the foundation of Western music for centuries—it's not cultural accident, but mathematical necessity!

## Link DataFrames: Relationships Between Sets

All link building functions are imported from `atonal.base`. Each link dataframe has columns:
- `source`: id of the first set
- `target`: id of the second set
- (optionally) additional metadata about the relationship

In [22]:
# All scalar and boolean predicates have been moved to atonal.base
# This cell is now empty and can be deleted or used for future exploration utilities

print("✓ All link building functions are now in atonal.base")

✓ All link building functions are now in atonal.base


In [23]:
# Import all link builders from atonal.base
from atonal.base import (
    build_immediate_subset_links_df,
    build_complement_links_df,
    build_ti_equivalence_links_df,
    build_z_relation_links_df,
    build_k_kh_links_df,
    build_rp_similarity_links_df,
)

print("✓ All link generators imported from atonal.base.")

✓ All link generators imported from atonal.base.


### Generate Link DataFrames

Let's generate several link dataframes. Note: Some computations are expensive for all 4096×4096 pairs, so we'll use optimizations.

In [24]:
# Generate the main link dataframes using imported builders
print("Building link dataframes...")

# 1. Immediate subset links (Hasse diagram of subset lattice)
print("  Building immediate subset links...")
immediate_subset_links = build_immediate_subset_links_df()
print(f"    {len(immediate_subset_links)} edges")

# 2. Complement links
print("  Building complement links...")
complement_links = build_complement_links_df()
print(f"    {len(complement_links)} edges")

# 3. TI-equivalence links (same set class)
print("  Building TI-equivalence links...")
ti_equiv_links = build_ti_equivalence_links_df(nodes_df)
print(f"    {len(ti_equiv_links)} edges")

# 4. Z-relation links
print("  Building Z-relation links...")
z_relation_links = build_z_relation_links_df(nodes_df)
print(f"    {len(z_relation_links)} edges")

print("\nDone!")

Building link dataframes...
  Building immediate subset links...
    24576 edges
  Building complement links...
    2048 edges
  Building TI-equivalence links...
    40594 edges
  Building Z-relation links...
    8796 edges

Done!


## Visualization with Force-Directed Graph

Let's visualize the subset lattice using a force-directed graph.

You'll need `cosmograph` for this part. To get it: `pip install cosmograph`

In [25]:
from cosmograph import cosmo

In [26]:
# Visualize with cosmograph - the subset lattice Hasse diagram
print("Visualizing subset lattice with cosmograph...")
g1 = cosmo(
    points=nodes_df,
    links=immediate_subset_links,
    point_id_by='id_',
    link_source_by='source',
    link_target_by='target',
    point_size_by='cardinality',
    point_color_by='cardinality',
)


Visualizing subset lattice with cosmograph...


(85968, 3)

## Additional Link Types

Let's add more sophisticated relationship links.

In [27]:
# Import additional link builders from atonal.base
from atonal.base import (
    build_k_kh_links_df,
    build_rp_similarity_links_df,
)

# ---------------------------------------------------------------------------
# Build K/Kh and R_p links using imported builders
# ---------------------------------------------------------------------------

print("Building K-complex links for sets with cardinality 3-9...")
forte_range_df = nodes_df[(nodes_df['cardinality'] >= 3) & (nodes_df['cardinality'] <= 9)]
print(f"  Working with {len(forte_range_df)} sets")

# You can build K or Kh links like this:
# kh_links = build_k_kh_links_df(forte_range_df, kh_only=True)
# k_links = build_k_kh_links_df(forte_range_df, kh_only=False)

print("\nBuilding R_p similarity links for triads (cardinality 3)...")
rp_triads = build_rp_similarity_links_df(nodes_df, cardinality=3)
print(f"  R_p triad links: {len(rp_triads)}")

print("\nBuilding R_p similarity links for tetrads (cardinality 4)...")
rp_tetrads = build_rp_similarity_links_df(nodes_df, cardinality=4)
print(f"  R_p tetrad links: {len(rp_tetrads)}")

print("\n✓ Link builders imported and used from atonal.base")

Building K-complex links for sets with cardinality 3-9...
  Working with 3938 sets

Building R_p similarity links for triads (cardinality 3)...
  R_p triad links: 19872

Building R_p similarity links for tetrads (cardinality 4)...
  R_p tetrad links: 85968

✓ Link builders imported and used from atonal.base


## Summary of Available Data

Let's review what we've built:

In [28]:
print("=" * 60)
print("NODES DATAFRAME")
print("=" * 60)
print(f"Total rows: {len(nodes_df)}")
print(f"\nColumns: {list(nodes_df.columns)}")
print(f"\nSample rows:")
display(nodes_df[nodes_df['cardinality'].isin([3, 4])].head(10))

print("\n" + "=" * 60)
print("LINK DATAFRAMES")
print("=" * 60)

link_summary = {
    'immediate_subset_links': immediate_subset_links,
    'complement_links': complement_links,
    'ti_equiv_links': ti_equiv_links,
    'z_relation_links': z_relation_links,
    'rp_triads': rp_triads,
    'rp_tetrads': rp_tetrads,
}

for name, df in link_summary.items():
    print(f"\n{name}: {len(df)} edges")
    if len(df) > 0:
        print(f"  Columns: {list(df.columns)}")

NODES DATAFRAME
Total rows: 4096

Columns: ['id_', 'pcset', 'cardinality', 'contains_zero', 'complement_id', 'prime_form', 'forte_name', 'is_forte_set', 'interval_vector', 'is_t_symmetric', 'z_correspondent_prime_form', 'z_correspondent_forte_name', 'n_T', 'n_I', 'kh_size', 'hexachord_combinatorial', 'max_T_invariance', 'max_T_invariance_n', 'max_I_invariance', 'max_I_invariance_n', 'best_normal_order']

Sample rows:


,id_,pcset,cardinality,contains_zero,complement_id,prime_form,forte_name,is_forte_set,interval_vector,is_t_symmetric,...,z_correspondent_forte_name,n_T,n_I,kh_size,hexachord_combinatorial,max_T_invariance,max_T_invariance_n,max_I_invariance,max_I_invariance_n,best_normal_order
7,7,"(0, 1, 2)",3,True,4088,"(0, 1, 2)",3-1,True,"(2, 1, 0, 0, 0, 0)",False,...,None,12,12,2,None,2,1,3,2,"(0, 1, 2)"
11,11,"(0, 1, 3)",3,True,4084,"(0, 1, 3)",3-2,True,"(1, 1, 1, 0, 0, 0)",False,...,None,12,12,2,None,1,1,2,1,"(0, 1, 3)"
13,13,"(0, 2, 3)",3,True,4082,"(0, 1, 3)",3-2,False,"(1, 1, 1, 0, 0, 0)",False,...,None,12,12,2,None,1,1,2,2,"(0, 2, 3)"
14,14,"(1, 2, 3)",3,False,4081,"(0, 1, 2)",3-1,False,"(2, 1, 0, 0, 0, 0)",False,...,None,12,12,2,None,2,1,3,4,"(1, 2, 3)"
15,15,"(0, 1, 2, 3)",4,True,4080,"(0, 1, 2, 3)",4-1,True,"(3, 2, 1, 0, 0, 0)",False,...,None,12,12,2,None,3,1,4,3,"(0, 1, 2, 3)"
19,19,"(0, 1, 4)",3,True,4076,"(0, 1, 4)",3-3,True,"(1, 0, 1, 1, 0, 0)",False,...,None,12,12,2,None,1,1,2,1,"(0, 1, 4)"
21,21,"(0, 2, 4)",3,True,4074,"(0, 2, 4)",3-6,True,"(0, 2, 0, 1, 0, 0)",False,...,None,12,12,2,None,2,2,3,4,"(0, 2, 4)"
22,22,"(1, 2, 4)",3,False,4073,"(0, 1, 3)",3-2,False,"(1, 1, 1, 0, 0, 0)",False,...,None,12,12,2,None,1,1,2,3,"(1, 2, 4)"
23,23,"(0, 1, 2, 4)",4,True,4072,"(0, 1, 2, 4)",4-2,True,"(2, 2, 1, 1, 0, 0)",False,...,None,12,12,2,None,2,1,3,2,"(0, 1, 2, 4)"
25,25,"(0, 3, 4)",3,True,4070,"(0, 1, 4)",3-3,False,"(1, 0, 1, 1, 0, 0)",False,...,None,12,12,2,None,1,1,2,3,"(0, 3, 4)"



LINK DATAFRAMES

immediate_subset_links: 24576 edges
  Columns: ['source', 'target']

complement_links: 2048 edges
  Columns: ['source', 'target']

ti_equiv_links: 40594 edges
  Columns: ['source', 'target']

z_relation_links: 8796 edges
  Columns: ['source', 'target', 'interval_vector', 'prime_form_a', 'prime_form_b']

rp_triads: 19872 edges
  Columns: ['source', 'target', 'max_common']

rp_tetrads: 85968 edges
  Columns: ['source', 'target', 'max_common']


## Interactive Exploration with Cosmograph

Choose different link types to visualize:

In [29]:
# Add human-readable label for visualization
nodes_df['label'] = nodes_df.apply(
    lambda r: f"{r['forte_name'] or ''} {r['pcset']}" if r['cardinality'] <= 6 else str(r['pcset']),
    axis=1
)

# Visualization function
def visualize_links(
    links_df: pd.DataFrame,
    title: str = "PC-Set Network",
    filter_cardinality: tuple = None,
    **cosmo_kwargs
):
    """
    Visualize a link dataframe with cosmograph.
    
    Args:
        links_df: Links with 'source' and 'target' columns
        title: Title for the visualization
        filter_cardinality: Optional (min, max) to filter nodes
        **cosmo_kwargs: Additional args passed to cosmo()
    """
    # Get nodes involved in these links
    involved_ids = set(links_df['source']) | set(links_df['target'])
    
    # Filter nodes
    vis_nodes = nodes_df[nodes_df['id_'].isin(involved_ids)].copy()
    
    if filter_cardinality:
        min_c, max_c = filter_cardinality
        vis_nodes = vis_nodes[(vis_nodes['cardinality'] >= min_c) & (vis_nodes['cardinality'] <= max_c)]
        involved_ids = set(vis_nodes['id_'])
        links_df = links_df[links_df['source'].isin(involved_ids) & links_df['target'].isin(involved_ids)]
    
    print(f"{title}: {len(vis_nodes)} nodes, {len(links_df)} edges")
    
    defaults = dict(
        points=vis_nodes,
        links=links_df,
        point_id_by='id_',
        link_source_by='source',
        link_target_by='target',
        point_size_by='cardinality',
        point_color_by='cardinality',
        point_label_by='label',
    )
    defaults.update(cosmo_kwargs)
    
    return cosmo(**defaults)


# Example visualizations:

print("Available visualizations:")
print("  1. visualize_links(immediate_subset_links, 'Subset Lattice')")
print("  2. visualize_links(ti_equiv_links, 'TI-Equivalence Classes')")
print("  3. visualize_links(complement_links, 'Complement Pairs')")
print("  4. visualize_links(z_relation_links, 'Z-Relations')")

Available visualizations:
  1. visualize_links(immediate_subset_links, 'Subset Lattice')
  2. visualize_links(ti_equiv_links, 'TI-Equivalence Classes')
  3. visualize_links(complement_links, 'Complement Pairs')
  4. visualize_links(z_relation_links, 'Z-Relations')


In [30]:
g2 = visualize_links(ti_equiv_links, "TI-Equivalence Classes", filter_cardinality=(3, 6))
g2

TI-Equivalence Classes: 2431 nodes, 24493 edges


Cosmograph(background_color=None, components_display_state_mode=None, focused_point_ring_color=None, hovered_p…

## Save the data to parquet files

In [ ]:
import os 

os.makedirs('tables', exist_ok=True)
os.makedirs('tables/pitch_class_sets', exist_ok=True)
nodes_df.to_parquet('tables/pitch_class_sets/twelve_tone_sets.parquet')

In [ ]:
# save the link data
os.makedirs('tables/pitch_class_sets/links', exist_ok=True)
for name, df in link_summary.items():
    df.to_parquet(f'tables/{name}.parquet')